In [4]:
library(Seurat)
library(sctransform)
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))
library(DoubletFinder)

Attaching SeuratObject



In [12]:
library(future)
library(future.apply)
plan("multicore", workers = 8) 
options(future.globals.maxSize = 100 * 1024^3)

In [7]:
rawdir <- paste0(getwd(), '/data/')
tmpdir <- paste0(getwd(), '/tmp/')

In [8]:
samples <- list.files(rawdir)
samples

[1] "GSM4929845_NPC_SC_1802_PBMC_count.csv.gz" 
 [2] "GSM4929846_NPC_SC_1802_Tumor_count.csv.gz"
 [3] "GSM4929847_NPC_SC_1805_PBMC_count.csv.gz" 
 [4] "GSM4929848_NPC_SC_1805_Tumor_count.csv.gz"
 [5] "GSM4929849_NPC_SC_1806_PBMC_count.csv.gz" 
 [6] "GSM4929850_NPC_SC_1806_Tumor_count.csv.gz"
 [7] "GSM4929851_NPC_SC_1807_PBMC_count.csv.gz" 
 [8] "GSM4929852_NPC_SC_1807_Tumor_count.csv.gz"
 [9] "GSM4929853_NPC_SC_1808_PBMC_count.csv.gz" 
[10] "GSM4929854_NPC_SC_1808_Tumor_count.csv.gz"
[11] "GSM4929855_NPC_SC_1810_PBMC_count.csv.gz" 
[12] "GSM4929856_NPC_SC_1810_Tumor_count.csv.gz"
[13] "GSM4929857_NPC_SC_1811_PBMC_count.csv.gz" 
[14] "GSM4929858_NPC_SC_1811_Tumor_count.csv.gz"
[15] "GSM4929859_NPC_SC_1813_PBMC_count.csv.gz" 
[16] "GSM4929860_NPC_SC_1813_Tumor_count.csv.gz"
[17] "GSM4929861_NPC_SC_1815_PBMC_count.csv.gz" 
[18] "GSM4929862_NPC_SC_1815_Tumor_count.csv.gz"
[19] "GSM4929863_NPC_SC_1816_PBMC_count.csv.gz" 
[20] "GSM4929864_NPC_SC_1816_Tumor_count.csv.gz"

In [19]:
## 批量导入数据，识别Doublet
starttime <- Sys.time()
slist <- future_lapply(samples, function(i){
  idir <- file.path(rawdir, i)
  a <- fread(idir, data.table = F) %>% column_to_rownames('V1')
  idat <- CreateSeuratObject(counts = a, min.cells = 3, min.features = 500, project = strsplit(i,'[.]')[[1]][1])
  idat[["percent.mt"]] <- PercentageFeatureSet(idat, pattern = "^MT-")
  idat <- subset(idat, subset = nFeature_RNA > 200 & nFeature_RNA < 25000 & percent.mt < 25 & nCount_RNA > 1000 & nCount_RNA < 500000)
  
  idat@meta.data$orig.ident <- paste(strsplit((idat@project.name),"_")[[1]][c(4,5)], collapse = '_')
  idat@meta.data$Patient <- strsplit((idat@project.name),"_")[[1]][4]
  idat@meta.data$Source <- strsplit((idat@project.name),"_")[[1]][5]
  idat@meta.data$barcode <- rownames(idat@meta.data)
  
  idat <- SCTransform(idat, method = "glmGamPoi", vars.to.regress = c("percent.mt"), verbose = F)
  idat <- RunPCA(idat, features = VariableFeatures(object = idat), verbose = F)
  pc.num = 1:30
  idat <- FindNeighbors(idat, dims = pc.num, verbose = F)
  idat <- FindClusters(idat, resolution = 0.5, verbose = F)
  idat <- RunTSNE(idat, dims = pc.num, reduction = "pca")
  # idat <- RunUMAP(idat, dims = pc.num, verbose = F)
  
  # 寻找最优pK值
  idat_db <- idat  # 用新变量跑DoubletFinder
  sweep.res.list_idat <- paramSweep_v3(idat_db, PCs = pc.num, sct = TRUE)
  sweep.stats_idat<- summarizeSweep(sweep.res.list_idat, GT = F)
  bcmvn_idat <- find.pK(sweep.stats_idat)
  pK_bcmvn <- bcmvn_idat$pK[which.max(bcmvn_idat$BCmetric)] %>% as.character() %>% as.numeric()
  
  # 排除不能检出的同源doublets，优化期望的doublets数量
  DoubletRate = 0.05    # 5000细胞对应的doublets rate是3.9%，见https://cloud.tencent.com/developer/article/1825672
  homotypic.prop <- modelHomotypic(idat_db@meta.data$seurat_clusters)   # 最好提供celltype（注释之后的？）
  nExp_poi <- round(DoubletRate*ncol(idat_db))
  nExp_poi.adj <- round(nExp_poi*(1-homotypic.prop))
  
  idat_db <- doubletFinder_v3(idat_db, PCs = pc.num, pN = 0.25, pK = pK_bcmvn, nExp = nExp_poi.adj, reuse.pANN = F, sct = TRUE)
  colnames(idat_db@meta.data)[grep('^DF',colnames(idat_db@meta.data))] <- "DF"
  
  idat$doublets <- idat_db$DF
  idat
})
endtime <- Sys.time()
endtime-starttime

In [ ]:
## 保存slist
saveRDS(slist, file = paste0(tmpdir, 'slist_sct_DF.Rds'))

In [ ]:
## 删除doublet
for (i in 1:length(slist)){
  slist[[i]] <- subset(slist[[i]], doublets == "Singlet")
}

In [ ]:
## 数据集整合（RPCA）
slist <- future_lapply(slist, SCTransform, method = "glmGamPoi")
features <- SelectIntegrationFeatures(slist, nfeatures = 3000)
slist <- PrepSCTIntegration(slist, anchor.features = features)
slist <- future_lapply(slist, RunPCA, features = features)

anchors <- FindIntegrationAnchors(slist, normalization.method = "SCT", reduction = "rpca",
                                  anchor.features = features, dims = 1:30, k.anchor = 20)
sce <- IntegrateData(anchors, normalization.method = "SCT", dims = 1:30)

In [ ]:
sce <- RunPCA(sce, verbose = F)
sce <- RunUMAP(sce, reduction = "pca", dims = 1:30)
sce <- FindNeighbors(sce, reduction = "pca", dims = 1:30)
sce <- FindClusters(sce, resolution = 1)

In [ ]:
## 保存sce
saveRDS(sce, file = paste0(tmpdir, 'sce_sct_rpca.Rds'))

In [ ]:
## 手动亚群注释
genes_to_check <- c('PTPRC', 'CD3D', 'CD3E','CD4', 'CD8A','FOXP3',  # Tcells
                    'CD19', 'CD79A', 'MS4A1' ,  # B cells
                    'FCGR3A', 'NCAM1',  # NK cells
                    'CD14',  'ITGAX',  # myeloid
                    'CD68',  'CD163',  # Mo/Mφ
                    'FGF7', 'MME','PECAM1', 'VWF',  # Fibroblasts, Endothelial
                    'EPCAM', 'KRT5'  # malignant
                    )
DotPlot(sce, features = unique(genes_to_check)) + coord_flip()

In [ ]:
cluster2celltype <- data.frame(cluster=levels(sce@meta.data$seurat_clusters), celltype='unknown')
cluster2celltype[cluster2celltype$cluster %in% c(0,3,8,10,11,14,20:22,26,28,32,34,35),2] <- 'CD8+T'
cluster2celltype[cluster2celltype$cluster %in% c(1,2,4,6,13,16,27,30,31,33),2] <- 'CD4+T'
cluster2celltype[cluster2celltype$cluster %in% c(5,9,12,23),2] <- 'NK'
cluster2celltype[cluster2celltype$cluster %in% c(7,15,17,18,29),2] <- 'Bcell'
cluster2celltype[cluster2celltype$cluster %in% c(19,25),2] <- 'Myeloid'
cluster2celltype[cluster2celltype$cluster %in% c(24),2] <- 'Malignant'
# cluster2celltype